<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Обработка-данных" data-toc-modified-id="Обработка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Обработка данных</a></span><ul class="toc-item"><li><span><a href="#Очистка-текста" data-toc-modified-id="Очистка-текста-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Очистка текста</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Разделение-на-выборки" data-toc-modified-id="Разделение-на-выборки-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Разделение на выборки</a></span></li></ul></li><li><span><a href="#Векторизация" data-toc-modified-id="Векторизация-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Векторизация</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#RandomForest" data-toc-modified-id="RandomForest-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>RandomForest</a></span></li><li><span><a href="#Catboost" data-toc-modified-id="Catboost-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Catboost</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
pip install pattern

Note: you may need to restart the kernel to use updated packages.


In [45]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pattern
from pattern.en import lemma
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from catboost import CatBoostClassifier
from sklearn.dummy import DummyClassifier

random_state = 12345

### Обзор данных

In [3]:
try:
    data = pd.read_csv('datasets\\toxic_comments.csv', index_col=[0])
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col=[0]) 

In [4]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Тексты на английском языке.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Пропусков нет

In [6]:
(data.toxic.value_counts() / data.shape[0]).round(2)

0    0.9
1    0.1
Name: toxic, dtype: float64

Очень сильный дисбаланс классов, это нужно будет учесть.

In [7]:
data.duplicated().sum()

0

Дубликатов нет

### Обработка данных

#### Очистка текста

In [8]:
# приводим весь текст к нижнему регистру
data['text'] = data['text'].str.lower()
# заменяем символ переноса строки на пробел
data['text'] = data['text'].str.replace('\n',' ')
# удаляем лишние символы
data['text'] = data['text'].apply((lambda x: re.sub("[^a-z0-9\s]",' ',x)))
#удаляем лишние пробелы
data['text'] = data['text'].apply((lambda x: " ".join(x.split())))
data.head(5)

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


Теперь отфильтруем текст от стоп-слов

In [9]:
stopwords_en = stopwords.words('english')
data['filtered_text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_en)]))
data.head()

,text,toxic,filtered_text
0,explanation why the edits made under my userna...,0,explanation edits made username hardcore metal...
1,d aww he matches this background colour i m se...,0,aww matches background colour seemingly stuck ...
2,hey man i m really not trying to edit war it s...,0,hey man really trying edit war guy constantly ...
3,more i can t make any real suggestions on impr...,0,make real suggestions improvement wondered sec...
4,you sir are my hero any chance you remember wh...,0,sir hero chance remember page


#### Лемматизация

In [10]:
try:
    print(lemma("I don't like exceptions"))
except:
    pass

In [11]:
data['filtered_text'] = data['filtered_text'].apply(lambda x: " ".join([lemma(wd) for wd in x.split()]))
data.head()

,text,toxic,filtered_text
0,explanation why the edits made under my userna...,0,explanation edit make username hardcore metall...
1,d aww he matches this background colour i m se...,0,aww match background colour seemingly stick th...
2,hey man i m really not trying to edit war it s...,0,hey man really try edit war guy constantly rem...
3,more i can t make any real suggestions on impr...,0,make real suggestion improvement wonder sectio...
4,you sir are my hero any chance you remember wh...,0,sir hero chance remember page


#### Разделение на выборки

Разделение на выборки проводим перед векторизацией.

In [12]:
train, test = train_test_split(data, test_size=0.2, random_state=random_state)

Валидационные данные пока отделять не будем. А тестовые данные отложим до финального тестирования.

### Векторизация

In [13]:
corpus = train['filtered_text'].values
count_tf_idf = TfidfVectorizer()
count_tf_idf.fit(corpus) 
tf_idf = count_tf_idf.transform(corpus)
tfidf_tokens = count_tf_idf.get_feature_names()

In [14]:
len(tfidf_tokens)

136800

Такое количество слов мы не можем обработать.  
Выделим словрь из 8000 самых популярных слов и отфильтруем данные по нему.

In [15]:
N = 8000
# индексы топ N столбцов с максимальной суммой элементов (в столбцах)
idx = np.ravel(tf_idf.sum(axis=0).argsort(axis=1))[::-1][:N]
top_N_words = np.array(count_tf_idf.get_feature_names())[idx].tolist()

In [16]:
train['filtered_text'] = train['filtered_text'].apply(lambda x: ' '.join([word for word in x.split() if word in (top_N_words)]))
train.head()

C:\Users\sergey_qt\AppData\Local\Temp\ipykernel_1604\3512392500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['filtered_text'] = train['filtered_text'].apply(lambda x: ' '.join([word for word in x.split() if word in (top_N_words)]))


,text,toxic,filtered_text
97494,bushranger you re a grass with no sense of hum...,1,grass sense humour see south park episode poor...
4383,need administrative help i have been blocked i...,0,need administrative help block read request tr...
103777,i d also like to point out that he has used a ...,0,also like point used third person plural prono...
38619,you cant block me you fucking retard brb nigger,1,cant block fuck retard nigger
128443,i believe that the frequency of the wave needs...,0,believe frequency wave need change 15 duration...


Теперь ещё раз проведём векторизацию

In [17]:
corpus = train['filtered_text'].values
count_tf_idf = TfidfVectorizer()
count_tf_idf.fit(corpus) 

tf_idf = count_tf_idf.transform(corpus)
tfidf_tokens = count_tf_idf.get_feature_names()

df_tfidfvect = pd.DataFrame(data = tf_idf.toarray(),columns = tfidf_tokens)
df_tfidfvect.head()

,00,000,000000,01,02,03,04,05,06,07,...,zealand,zero,zeu,zionism,zionist,zoe,zombie,zone,zoo,zora
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Обучение

In [18]:
x_train, x_valid, y_train, y_valid = train_test_split(df_tfidfvect, train['toxic'], test_size=0.2, random_state=random_state)

### LogisticRegression

Для устранения дисбаланса здесь и далее будем использовать взвешенные классы.

In [19]:
%%time
best_linear_model = None
best_iter = 0
best_linear_f1 = 0
for iter in range(100, 201, 50):
    linear_model = LogisticRegression(class_weight='balanced', max_iter=iter, random_state=random_state)
    linear_model.fit(x_train, y_train)
    linear_prediction = (linear_model.predict(x_valid))
    f1_score_linear = f1_score(y_valid, linear_prediction)
    if f1_score_linear > best_linear_f1:
        best_linear_model = linear_model
        best_iter = iter
        best_linear_f1 = f1_score_linear

C:\Users\sergey_qt\anaconda3\envs\practicum\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CPU times: total: 31min 14s
Wall time: 5min 33s


In [20]:
best_linear_f1, best_iter

(0.7199341021416804, 100)

In [21]:
probabilities_weighted = best_linear_model.predict_proba(x_valid)[:, 1]

best_linear_f1_threshold = 0
best_threshold = 0
for threshold in np.arange(0, 1, 0.01):
    predicted_valid = probabilities_weighted > threshold
    score = f1_score(y_valid, predicted_valid)
    if best_linear_f1_threshold < score:
        best_linear_f1_threshold = score
        best_linear_threshold = threshold
print('Лучший порог:',best_linear_threshold,'\nF1:', best_linear_f1_threshold)

Лучший порог: 0.76 
F1: 0.7696768848384425


### RandomForest

In [22]:
%%time
best_forest_model = None
best_est = 0
best_forest_f1 = 0
for est in range(100, 501, 200):
    forest_model = RandomForestClassifier(n_estimators=est, class_weight='balanced', random_state=random_state)
    forest_model.fit(x_train, y_train)
    forest_prediction = forest_model.predict(x_valid)
    f1_score_forest = f1_score(y_valid, forest_prediction)
    if f1_score_forest > best_forest_f1:
        best_forest_model = forest_model
        best_est = est
        best_forest_f1 = f1_score_forest

CPU times: total: 2h 28min 24s
Wall time: 2h 28min 44s


In [23]:
best_forest_f1, best_est

(0.6988610478359908, 300)

In [24]:
probabilities_weighted = best_forest_model.predict_proba(x_valid)[:, 1]

best_forest_f1_threshold = 0
best_forest_threshold = 0
for threshold in np.arange(0, 1, 0.01):
    predicted_valid = probabilities_weighted > threshold
    score = f1_score(y_valid, predicted_valid)
    if best_forest_f1_threshold < score:
        best_forest_f1_threshold = score
        best_forest_threshold = threshold
print('Лучший порог:',best_forest_threshold,'\nF1:', best_forest_f1_threshold)

Лучший порог: 0.33 
F1: 0.7352425482174166


### Catboost

In [32]:
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

In [33]:
%%time
best_cat_model = None
best_iter = 0
best_cat_f1 = 0
for iterations in range(500, 1501, 500):
    cat_model = CatBoostClassifier(iterations=iterations,class_weights=class_weights,random_seed=random_state)
    cat_model.fit(x_train, y_train, verbose=500)
    cat_prediction = cat_model.predict(x_valid)
    f1_score_cat = f1_score(y_valid, cat_prediction)
    if f1_score_cat > best_cat_f1:
        best_cat_model = cat_model
        best_iter = iterations
        best_cat_f1 = f1_score_cat

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.140137
0:	learn: 0.6288894	total: 1.5s	remaining: 12m 27s
499:	learn: 0.1916455	total: 3m 47s	remaining: 0us
Learning rate set to 0.074218
0:	learn: 0.6558981	total: 954ms	remaining: 15m 53s
500:	learn: 0.2433495	total: 3m 35s	remaining: 3m 34s
999:	learn: 0.1877493	total: 7m 6s	remaining: 0us
Learning rate set to 0.051172
0:	learn: 0.6666568	total: 1.25s	remaining: 31m 12s
500:	learn: 0.2730325	total: 3m 36s	remaining: 7m 12s
1000:	learn: 0.2170115	total: 7m 5s	remaining: 3m 32s
1499:	learn: 0.1847010	total: 10m 34s	remaining: 0us
CPU times: total: 2h 39min 39s
Wall time: 24min 18s


In [34]:
best_cat_f1, best_iter

(0.7536646406864498, 1500)

In [37]:
probabilities_weighted = best_cat_model.predict_proba(x_valid)[:, 1]

best_cat_f1_threshold = 0
best_cat_threshold = 0
for threshold in np.arange(0, 1, 0.01):
    predicted_valid = probabilities_weighted > threshold
    score = f1_score(y_valid, predicted_valid)
    if best_cat_f1_threshold < score:
        best_cat_f1_threshold = score
        best_cat_threshold = threshold
print('Лучший порог:',best_cat_threshold,'\nF1:', best_cat_f1_threshold)

Лучший порог: 0.67 
F1: 0.776321179844326


## Тестирование

In [38]:
print('LogisticRegression F1:', best_linear_f1_threshold)
print('RandomForest F1:', best_forest_f1_threshold)
print('CatBoost F1:', best_cat_f1_threshold)

LogisticRegression F1: 0.7696768848384425
RandomForest F1: 0.7352425482174166
CatBoost F1: 0.776321179844326


С небольшим перевесом победил CatBoost. Обучим его заново на объединении тренировочных и валидационных данных для лучшего результата. 

In [39]:
best_model = cat_model
best_threshold = best_cat_threshold

In [40]:
best_model.fit(df_tfidfvect, train['toxic'], verbose=100)

Learning rate set to 0.056288
0:	learn: 0.6629656	total: 1.68s	remaining: 41m 58s
100:	learn: 0.3905029	total: 50.9s	remaining: 11m 45s
200:	learn: 0.3424325	total: 1m 38s	remaining: 10m 39s
300:	learn: 0.3093738	total: 2m 26s	remaining: 9m 44s
400:	learn: 0.2868954	total: 3m 14s	remaining: 8m 52s
500:	learn: 0.2694899	total: 4m 1s	remaining: 8m 2s
600:	learn: 0.2551498	total: 4m 49s	remaining: 7m 13s
700:	learn: 0.2436057	total: 5m 37s	remaining: 6m 24s
800:	learn: 0.2331731	total: 6m 25s	remaining: 5m 36s
900:	learn: 0.2244824	total: 7m 12s	remaining: 4m 47s
1000:	learn: 0.2166047	total: 8m	remaining: 3m 59s
1100:	learn: 0.2092886	total: 8m 47s	remaining: 3m 11s
1200:	learn: 0.2027350	total: 9m 35s	remaining: 2m 23s
1300:	learn: 0.1966544	total: 10m 23s	remaining: 1m 35s
1400:	learn: 0.1912660	total: 11m 10s	remaining: 47.4s
1499:	learn: 0.1858623	total: 11m 57s	remaining: 0us


Векторизуем тестовые данные.

In [41]:
corpus_test = test['filtered_text'].values

tf_idf_test = count_tf_idf.transform(corpus_test)

df_tfidfvect_test = pd.DataFrame(data = tf_idf_test.toarray(),columns = tfidf_tokens)

Проверяем модель.

In [43]:
x_test = df_tfidfvect_test
y_test = test['toxic']

probabilities_test = best_model.predict_proba(x_test)[:, 1]
predicted_test = probabilities_test > best_threshold
test_score = f1_score(y_test, predicted_test)
test_score

0.7898457583547558

## Выводы

Выбранный вариант векторизации слов очень простой - он не учитывает порядок слов или их близость по смыслу. Однако нам удалось достигнуть поставленного порога метрики F1 в 0.75.  
Лучший результат показала следующая модель:  
**CatBoost**
* F1 = 0.79
* iterations = 1500